<a href="https://colab.research.google.com/github/kaganseyda/projects/blob/main/competetion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import Ridge
from scipy.sparse import hstack
import numpy as np
from sklearn.preprocessing import normalize
from sklearn.model_selection import GridSearchCV

In [ ]:
train_df = pd.read_csv('train.csv')
train_df.head()

In [ ]:
test_df = pd.read_csv('test.csv')
test_df.head()

In [6]:
stopwords=['i', 'me', 'my', 'myself', 'we', 'our', 'ours',
'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself',
'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself',
'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves',
 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are',
 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing',
 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for',
'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to',
'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once',
'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most',
'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very',
's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y',
 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't",
'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn',
 "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't",
'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]

for word in stopwords:
    word=" "+word+" "
    train_df['prompt']=train_df['prompt'].str.replace(word," ")
    test_df['prompt']=test_df['prompt'].str.replace(word," ")

In [7]:
vectorizer = TfidfVectorizer()

train_responses = pd.concat([train_df['model_a'], train_df['model_b'], train_df['prompt'], train_df['response_a'], train_df['response_b']])
vectorizer.fit(train_responses)

TfidfVectorizer()

In [36]:
vectorizer_prompt = TfidfVectorizer().fit(train_df['prompt'])
vectorizer_response_a = TfidfVectorizer().fit(train_df['response_a'])
vectorizer_response_b = TfidfVectorizer().fit(train_df['response_b'])
train_prompt = vectorizer_prompt.transform(train_df['prompt'])
train_response_a = vectorizer_response_a.transform(train_df['response_a'])
train_response_b = vectorizer_response_b.transform(train_df['response_b'])
train_responses = hstack([train_prompt, train_response_a, train_response_b])
param_grid = {'estimator__alpha': [0.1, 1.0, 10.0, 100.0]}
model = MultiOutputRegressor(Ridge())
grid_search = GridSearchCV(model, param_grid, cv=5)
train_labels = train_df[['winner_model_a', 'winner_model_b', 'winner_tie']]
grid_search.fit(train_responses, train_labels)


best_model = grid_search.best_estimator_
test_prompt = vectorizer_prompt.transform(test_df['prompt'])
test_response_a = vectorizer_response_a.transform(test_df['response_a'])
test_response_b = vectorizer_response_b.transform(test_df['response_b'])
test_responses = hstack([test_prompt, test_response_a, test_response_b])

predictions = best_model.predict(test_responses)
best_model=model


def softmax(x):
    e_x = np.exp(x - np.max(x, axis=1, keepdims=True))
    return e_x / e_x.sum(axis=1, keepdims=True)

predictions_normalized = softmax(predictions)


submission = pd.DataFrame(predictions_normalized, columns=['winner_model_a', 'winner_model_b', 'winner_tie'])
submission['id'] = test_df['id']


submission.to_csv('submission.csv', index=False)


print(submission.head())


   winner_model_a  winner_model_b  winner_tie       id
0        0.296295        0.327439    0.376266   136060
1        0.355792        0.328097    0.316111   211333
2        0.344718        0.341714    0.313569  1233961


In [39]:
model.score(train_responses, train_labels)

0.08073773646254019